In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import numpy as np

In [ ]:
from pkg.tf_transform import *
from pkg.tf_robot import *
from pkg.constraint import *
from pkg.info import *
from pkg.tf_utils import *
from pkg.rotation_utils import *

In [ ]:
import matplotlib.pyplot as plt
import time

# Load urdf 

In [ ]:
from urdf_parser_py.urdf import URDF
from pkg.ur10 import URDF_PATH, JOINT_NAMES, LINK_NAMES, ZERO_JOINT_POSE
from pkg.joint_utils import *
urdf_content = URDF.from_xml_file(URDF_PATH)

In [ ]:
link_names = LINK_NAMES
base_name = LINK_NAMES[0]
joint_names = JOINT_NAMES
link_info_list = get_link_info_list(link_names,base_name, urdf_content)

# Visualize

In [ ]:
from pkg.ros_rviz import *
pub, joints, rate = get_publisher(JOINT_NAMES)

# Constraint

# Graph

In [ ]:
class GraphModel(tf.keras.Model):

    def __init__(self, robot_info_list, N_sim, alpha_jc=1, alpha_fc=100):
        super(GraphModel, self).__init__()
        self.robot_list = []
        self.jc_list = []
        self.fc_list = []
        self.robot_base_list = []
        self.alpha_jc = alpha_jc
        self.alpha_fc = alpha_fc
        for robot_info in robot_info_list:
            robot = RobotLayer(
                robot_info.link_info_list, rname = robot_info.rname, base_name=robot_info.base_name, dim=N_sim)
            self.robot_list += [robot]
            self.jc_list += [JointConstraintLoss(robot)]
            self.fc_list += [FrameConstraintLoss()]
            self.robot_base_list += [[np.identity(4,dtype=np.float32)]*N_sim]
            
    def assign_Q_list(self, Q_list):
        for robot, Q in zip(self.robot_list, Q_list):
            robot.assign_Q(Q)
            
    def get_Q_list(self):
        Q_list = []
        for robot in self.robot_list:
            Q_list += [robot.get_Q()]
        return Q_list

    def call(self, inputs_list):
        T_all_list = []
        jc_list = []
        fc_list = []
        for robot, robot_base, jc_loss, fc_loss, inputs in zip(
                self.robot_list, self.robot_base_list, 
                self.jc_list, self.fc_list, inputs_list):
            T_all = robot(robot_base)
            T_all_list += [T_all]
            jc_list += [jc_loss(inputs[0:2])]
            fc_list += [fc_loss((T_all[:,-1,:,:],)+inputs[2:4])]
            
        return self.alpha_jc*K.sum(jc_list)+self.alpha_fc*K.sum(fc_list)


In [ ]:
N_sim = 50
N_joints = 8
DOF = 6
graph = GraphModel([RobotInfo(link_info_list, rname = "rbt1", base_name=base_name)], N_sim=N_sim)

In [ ]:
learning_rate = 1e-1
# Stochastic gradient descent optimizer.
# optimizer = tf.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.99, amsgrad=False)
# optimizer = tf.optimizers.Adagrad(learning_rate*5)
optimizer = tf.optimizers.SGD(learning_rate)
# optimizer = tf.optimizers.RMSprop(learning_rate)

In [ ]:
Q_ = np.array([(0,)+tuple((np.random.rand(DOF)*2-1)*np.pi)+(0,) for _ in range(N_sim)], dtype=np.float32)
Ttar = SE3(Rot_zyx(0,0,np.pi),(0.5,0,0)).astype(np.float32)
graph.assign_Q_list([Q_])
time_vec = []
Q_list_vec = [list(map(lambda x: x.numpy(), graph.get_Q_list()))]
N_iter = 50
# Run training for the given number of steps.
for _ in range(N_iter):
    # Run the optimization to update W and b values.
    t1 = time.time()
    with tf.GradientTape() as g:
        # Forward pass.
        loss = graph([(np.zeros((N_sim, N_joints), dtype='float32'), np.zeros(N_sim, dtype='float32'),
                       np.array([Ttar]*N_sim), np.ones(N_sim, dtype='float32')
                      )])
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = graph.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    gradients = clip_gradient(gradients, 1)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))
    t2 = time.time()
    time_vec += [t2-t1]
    Q_list_vec += [list(map(lambda x: x.numpy(), graph.get_Q_list()))]
print("mean {} ms".format(round(np.mean(time_vec)*1000,2)))
print("total {} ms".format(round(np.sum(time_vec)*1000,2)))

In [ ]:
i_rbt = 0
for i_sim in range(1): # N_sim):
    for i_iter in range(N_iter):
        q = Q_list_vec[i_iter][i_rbt][i_sim]
        pose_list = [q[1:7]]
        show_motion(pose_list, [], pub, joints, joint_names, error_skip=1e-6, period=1e-6)
        time.sleep(1e-1)

In [ ]:
Q_all = np.array(Q_list_vec)
Q_all_i = np.array([Q_all[i_iter, i_rbt, i_sim,:] for i_iter in range(N_iter)])
plt.plot(Q_all_i)

In [ ]:
plt.plot(Q_all_i[1:,:]-Q_all_i[:-1,:])

In [ ]:
# N_r = 100
# DOF = 6
# robot = RobotLayer(link_info_list, rname = "rb1", base_name=base_name, dim=N_r)
# qvec = []
# Pvec_list = []
# time_list = []
# asg_time_list = []
# for _ in range(100):
#     q = np.array([(0,)+tuple((np.random.rand(DOF)*2-1)*np.pi/3)+(0,) for _ in range(N_r)], dtype=np.float32)
#     qvec += [q]
#     t0 = time.time()
#     robot.assign_Q(q)
#     t01 = time.time()
#     T_all = robot([np.identity(4,dtype=np.float32)]*N_r)
#     t1 = time.time()
#     Pvec = np.array(T_all[:,:,:3,3])
#     Pvec_list += list(Pvec)
#     time_list += [t1-t0]
#     asg_time_list += [t01-t0]
# Pvec_list=np.array(Pvec_list)
# print("assign average: {} ms".format(np.mean(asg_time_list)*1000))
# print("total average: {} ms".format(np.mean(time_list)*1000))
# # for Pvec in Pvec_list:
# #     plt.plot(Pvec[:,0],Pvec[:,1],'-o')
# # plt.axis('equal')

In [ ]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow.keras import Model, layers
import numpy as np

In [ ]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.1
training_steps = 2000
batch_size = 50
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

In [ ]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [ ]:

# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [ ]:

# Create TF Model.
class NeuralNet(Model):
    # Set layers.
    def __init__(self):
        super(NeuralNet, self).__init__()
        # First fully-connected hidden layer.
        self.fc1 = layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # First fully-connected hidden layer.
        self.fc2 = layers.Dense(n_hidden_2, activation=tf.nn.relu)
        # Second fully-connecter hidden layer.
        self.out = layers.Dense(num_classes)

    # Set forward pass.
    def call(self, x, is_training=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        if not is_training:
            # tf cross entropy expect logits without softmax, so only
            # apply softmax when not training.
            x = tf.nn.softmax(x)
        return x

# Build neural network model.
neural_net = NeuralNet()

In [ ]:

# Cross-Entropy Loss.
# Note that this will apply 'softmax' to the logits.
def cross_entropy_loss(x, y):
    # Convert labels to int 64 for tf cross-entropy function.
    y = tf.cast(y, tf.int64)
    # Apply softmax to logits and compute cross-entropy.
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    # Average loss across the batch.
    return tf.reduce_mean(loss)

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [ ]:
# Optimization process. 
def run_optimization(x, y):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as g:
        # Forward pass.
        pred = neural_net(x, is_training=True)
        # Compute loss.
        loss = cross_entropy_loss(pred, y)
        
    # Variables to update, i.e. trainable variables.
    trainable_variables = neural_net.trainable_variables

    # Compute gradients.
    gradients = g.gradient(loss, trainable_variables)
    
    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [ ]:
import time

In [ ]:
time_vec = []
# Run training for the given number of steps.
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    # Run the optimization to update W and b values.
    t1 = time.time()
    run_optimization(batch_x, batch_y)
    t2 = time.time()
    time_vec += [t2-t1]
    
    if step % display_step == 0:
        pred = neural_net(batch_x, is_training=True)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

In [ ]:
np.mean(time_vec)*100